#Segmenting and Clustering Neighborhoods in Toronto. 

##All 3 sections of the assignment is completed in the same notebook. Please scroll down to see the solutions.

#1. Creating dataframe from WikiPedia page using the pandas read_html

In [0]:
#Importing necessary libraris

import pandas as pd
import numpy as np
import csv

In [0]:
#reading the data. the na values are already considered as Not Assigned hence they will be not included

html=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', na_values='Not assigned', flavor='html5lib')
print(type(html[0]))
df=html[0]
#check the dataframe
print(df.columns)
#check the data
df.head()

<class 'pandas.core.frame.DataFrame'>
Index(['Postal code', 'Borough', 'Neighborhood'], dtype='object')


,Postal code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [0]:
#dealing with NaN 
print(df.isnull().sum())
df.drop(df.index[0], inplace = True)
df.dropna(axis = 0, inplace = True)
print(df.isnull().sum())

#to know the number of rows in the dataframe
print(df.shape)

Postal code      0
Borough         77
Neighborhood    77
dtype: int64
Postal code     0
Borough         0
Neighborhood    0
dtype: int64
(103, 3)


#2. Getting the coordinates using geocoder

In [0]:
pip install geocoder

In [0]:
#importing libraries
import geocoder
import time

In [0]:
#getting postal codes
df['Postal code'].unique()

array(['M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M9A', 'M1B', 'M3B', 'M4B',
       'M5B', 'M6B', 'M9B', 'M1C', 'M3C', 'M4C', 'M5C', 'M6C', 'M9C',
       'M1E', 'M4E', 'M5E', 'M6E', 'M1G', 'M4G', 'M5G', 'M6G', 'M1H',
       'M2H', 'M3H', 'M4H', 'M5H', 'M6H', 'M1J', 'M2J', 'M3J', 'M4J',
       'M5J', 'M6J', 'M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M1L',
       'M2L', 'M3L', 'M4L', 'M5L', 'M6L', 'M9L', 'M1M', 'M2M', 'M3M',
       'M4M', 'M5M', 'M6M', 'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N',
       'M6N', 'M9N', 'M1P', 'M2P', 'M4P', 'M5P', 'M6P', 'M9P', 'M1R',
       'M2R', 'M4R', 'M5R', 'M6R', 'M7R', 'M9R', 'M1S', 'M4S', 'M5S',
       'M6S', 'M1T', 'M4T', 'M5T', 'M1V', 'M4V', 'M5V', 'M8V', 'M9V',
       'M1W', 'M4W', 'M5W', 'M8W', 'M9W', 'M1X', 'M4X', 'M5X', 'M8X',
       'M4Y', 'M7Y', 'M8Y', 'M8Z'], dtype=object)

In [0]:
#sort the dataframe and drop its index

df = df.sort_values(by = ['Postal code'], ascending = True)
df.reset_index(inplace= True, drop = True)
df.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [0]:
latitude = []
longitude = []
postal_code = df['Postal code']

for j in range(len(df)):
    print(j)
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code[j]))
        lat_lng_coords = g.latlng

    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
    
df['Latitude'], df['Longitude'] = latitude, longitude

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102


In [0]:
df['Neighborhood']=df['Neighborhood'].str.replace('/', ',')
df.dropna(axis=0)
df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.808626,-79.189913
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.785779,-79.157368
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813


# 3. Explore and cluster the neighbourhoods in Toronto

In [0]:
#import libraris
import numpy as np
import pandas as pd
import requests
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print('All libraries imported')

All libraries imported


In [0]:
#count the boroughs and neighbourhoods

print('The dataframe has {} boroughs and {} neighbourhoods.' .format(len(df['Borough'].unique()), len(df['Neighborhood'].unique()) ))

The dataframe has 10 boroughs and 98 neighbourhoods.


In [0]:
#let's create a map of Toronto using the latitude and longitude values
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

map_toronto=folium.Map(location=[latitude, longitude], zoom_start=10)

#add markers to map 
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [0]:
#lets segment and cluster only the neighborhoods with Borogh consisting of name Toronto. 
#Slice the original dataframe and create a new dataframe of Toronot Borough.

toronto_data=df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.678148,-79.295349
1,M4K,East Toronto,"The Danforth West , Riverdale",43.683424,-79.354564
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.668291,-79.315578
3,M4M,East Toronto,Studio District,43.648000,-79.339260
4,M4N,Central Toronto,Lawrence Park,43.729455,-79.386415


In [0]:
# create map of toronto neighbourhood using latitude and longitude values
map_toronto_new = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_new)  
    
map_toronto_new

In [0]:
#Now using the foursquare API, we explore the neighbourhoods and segment them. 

CLIENT_ID = 'I5HYQK2UM15VPOYZJPJHCAIWNNDGEUBNPWLTIPDQ4SAUI43E' # your Foursquare ID
CLIENT_SECRET = 'YLXU4UZGUC54EXYOL5VJXV5JUYUOYWXUJMSESDNA5I1GA42R' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: I5HYQK2UM15VPOYZJPJHCAIWNNDGEUBNPWLTIPDQ4SAUI43E
CLIENT_SECRET:YLXU4UZGUC54EXYOL5VJXV5JUYUOYWXUJMSESDNA5I1GA42R


In [0]:
#let's explore the first neighborhood in our dataframe
#get the name of the neighborhood
toronto_data.loc[0, 'Neighborhood']

'The Beaches'

In [0]:
#get the neighborhood's latitude and longitude
neighborhood_latitude=toronto_data.loc[0, 'Latitude']
neighborhood_longitude=toronto_data.loc[0,'Longitude']
neighborhood_name=toronto_data.loc[0,'Neighborhood']
print('Latitude and Longitude values of {} are {}, {}' .format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and Longitude values of The Beaches are 43.67814827600006, -79.29534930999995


In [0]:
#now, let's get the top 100 venues that are in The Beaches within a radius of 500 metres using FourSquare API. 
LIMIT= 100 #limit number of venues returned by Foursquare API
radius=500 #we need to define radius of return
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
results=requests.get(url).json()
results



{'meta': {'code': 200, 'requestId': '5ead2c2daba297001b2c51fd'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bd461bc77b29c74a07d9282-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/hikingtrail_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d159941735',
         'name': 'Trail',
         'pluralName': 'Trails',
         'primary': True,
         'shortName': 'Trail'}],
       'id': '4bd461bc77b29c74a07d9282',
       'location': {'address': 'Glen Manor',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Queen St.',
        'distance': 186,
        'formattedAddress': ['Glen Manor (Queen St.)', 'Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.6768

In [0]:
#function that extracts the category of the venue
def get_category_type(row):
  try:
    categories_list=row['categories']
  except:
    categories_list=row['venue.categories']
  
  if len(categories_list)==0: 
    return None
  else: 
    return categories_list[0]['name']

In [0]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#Now we are ready to clean the json and structure it into a *pandas* dataframe.
venues = results['response']['groups'][0]['items']
   
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [0]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [0]:
#lets create a function to repeat the same process to all neighbourhoods in Manhatten
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        results
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [0]:
# type your answer here

toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West , Riverdale
India Bazaar , The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park , Summerhill East
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
Rosedale
St. James Town , Cabbagetown
Church and Wellesley
Regent Park , Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond , Adelaide , King
Harbourfront East , Union Station , Toronto Islands
Toronto Dominion Centre , Design Exchange
Commerce Court , Victoria Hotel
Roselawn
Forest Hill North & West
The Annex , North Midtown , Yorkville
University of Toronto , Harbord
Kensington Market , Chinatown , Grange Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Stn A PO Boxes
First Canadian Place , Underground city
Christie
Dufferin , Dovercourt Village
Little Portugal , Trinity
Brockton , Parkdale Village , Exhibition Place
High Park , 

In [0]:
#check the size of the resulting dataframe
print(toronto_venues.shape)

(1598, 7)


In [0]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.678148,-79.295349,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.678148,-79.295349,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.678148,-79.295349,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.678148,-79.295349,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West , Riverdale",43.683424,-79.354564,Centennial College - Story Arts Centre,43.684874,-79.349140,Business Service


In [0]:
#check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,66,66,66,66,66,66
"Brockton , Parkdale Village , Exhibition Place",43,43,43,43,43,43
Business reply mail Processing CentrE,100,100,100,100,100,100
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",64,64,64,64,64,64
Central Bay Street,54,54,54,54,54,54
Christie,12,12,12,12,12,12
Church and Wellesley,86,86,86,86,86,86
"Commerce Court , Victoria Hotel",100,100,100,100,100,100
Davisville,26,26,26,26,26,26


In [0]:
#find out how many unique categories can be curated from all the returned venues
print('There are {} unique categories.' .format(len(toronto_venues['Venue Category'].unique())))

There are 221 unique categories.


In [0]:
#analyze each neighborhood
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood '] = toronto_venues['Neighborhood'] 
toronto_onehot['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,...,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West , Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#examine the shape of dataframe
toronto_onehot.shape

(1598, 222)

In [0]:
#let's group rows by neighborhood and take the mean of the frequency of occurence of each category 

toronto_grouped=toronto_onehot.groupby('Neighborhood ').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,...,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.015152,0.000000,0.000000,0.000000,0.000000,0.015152,0.000000,0.015152,0.030303,0.000000,0.000000,0.00,0.015152,0.000000,0.030303,0.000000,0.00000,0.000000,0.015152,0.000000,0.000000,0.0000,0.030303,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.030303,0.000000,0.000000,0.030303,0.000000,0.000000,...,0.000000,0.030303,0.000000,0.015152,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00000,0.015152,0.000000,0.015152,0.000000,0.000000,0.000000,0.000000,0.00,0.015152,0.000000,0.000000,0.000000,0.000000,0.000000,0.015152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.015152,0.000000,0.000000,0.000000,0.00,0.015152
1,"Brockton , Parkdale Village , Exhibition Place",0.023256,0.000000,0.023256,0.000000,0.023256,0.000000,0.000000,0.000000,0.000000,0.000000,0.023256,0.023256,0.023256,0.00,0.000000,0.023256,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.023256,0.0000,0.000000,0.023256,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.069767,0.000000,0.023256,0.000000,0.000000,0.023256,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.023256,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.023256,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046512,0.023256,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,Business reply mail Processing CentrE,0.000000,0.020000,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.020000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.020000,0.000000,0.0100,0.010000,0.000000,0.000000,0.00,0.000000,0.010000,0.000000,0.0,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.020000,0.010000,0.000000,0.000000,0.010000,0.000000,0.00,0.01,0.010000,0.01000,0.000000,0.000000,0.020000,0.000000,0.000000,0.020000,0.000000,0.02,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.020000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.00,0.000000
3,"CN Tower , King and Spadina , Railway Lands , ...",0.000000,0.000000,0.000000,0.000000,0.000000,0.015625,0.000000,0.000000,0.000000,0.000000,0.015625,0.015625,0.031250,0.00,0.000000,0.000000,0.015625,0.000000,0.00000,0.015625,0.000000,0.000000,0.000000,0.0000,0.000000,0.015625,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.062500,0.000000,0.015625,0.000000,0.000000,0.000000,...,0.000000,0.015625,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.015625,0.03125,0.000000,0.000000,0.000000,0.000000,0.000000,0.015625,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.018519,0

In [0]:
toronto_grouped.shape

(38, 222)

In [0]:
#print each neighborhood along with the top 5 common venues

num_top_venues=5

for hood in toronto_grouped['Neighborhood ']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood '] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n') 

----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.05
2            Café  0.03
3  Breakfast Spot  0.03
4      Restaurant  0.03


----Brockton , Parkdale Village , Exhibition Place----
                    venue  freq
0             Coffee Shop  0.09
1                    Café  0.07
2               Gift Shop  0.05
3  Thrift / Vintage Store  0.05
4                   Diner  0.02


----Business reply mail Processing CentrE----
                 venue  freq
0          Coffee Shop  0.07
1                Hotel  0.05
2  Japanese Restaurant  0.04
3                 Café  0.04
4           Restaurant  0.03


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
               venue  freq
0               Café  0.06
1        Coffee Shop  0.06
2  French Restaurant  0.05
3         Restaurant  0.05
4               Park  0.05


----Central Bay Street----
                       venue  freq
0               

In [0]:
## Let's put that into a *pandas* dataframe
## First, let's write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [0]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood ']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Breakfast Spot,Beer Bar,Seafood Restaurant,Lounge,Bakery,Café,Restaurant,Hotel
1,"Brockton , Parkdale Village , Exhibition Place",Coffee Shop,Café,Thrift / Vintage Store,Gift Shop,French Restaurant,Indian Restaurant,Boutique,Pizza Place,Brewery,Italian Restaurant
2,Business reply mail Processing CentrE,Coffee Shop,Hotel,Japanese Restaurant,Café,Restaurant,Asian Restaurant,Italian Restaurant,Concert Hall,Mediterranean Restaurant,Steakhouse
3,"CN Tower , King and Spadina , Railway Lands , ...",Café,Coffee Shop,French Restaurant,Restaurant,Park,Bar,Italian Restaurant,Japanese Restaurant,Lounge,Speakeasy
4,Central Bay Street,Coffee Shop,Café,Middle Eastern Restaurant,Bubble Tea Shop,Restaurant,Sandwich Place,Plaza,Italian Restaurant,Breakfast Spot,Clothing Store


#Cluster Neighborhood

In [0]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood ', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [0]:
#Let's create a new dataframe that includes the cluster as well as 
#the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.678148,-79.295349,0.0,Health Food Store,Trail,Pub,Neighborhood,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
1,M4K,East Toronto,"The Danforth West , Riverdale",43.683424,-79.354564,0.0,Bus Line,Business Service,Park,Grocery Store,Discount Store,Yoga Studio,Elementary School,Flower Shop,Fish Market,Fish & Chips Shop
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.668291,-79.315578,0.0,Fast Food Restaurant,Sandwich Place,Park,Liquor Store,Sushi Restaurant,Restaurant,Italian Restaurant,Pub,Steakhouse,Food & Drink Shop
3,M4M,East Toronto,Studio District,43.648000,-79.339260,0.0,Baseball Field,Athletics & Sports,Government Building,Night Market,Electronics Store,Food,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant
4,M4N,Central Toronto,Lawrence Park,43.729455,-79.386415,3.0,Construction & Landscaping,Bus Line,Swim School,Yoga Studio,Electronics Store,Food,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant


In [0]:
#Finally, let's visualize the resulting clusters
# create map

toronto_merged=toronto_merged.dropna(axis=0)
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1 ],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#lets analyze each cluster separately

In [0]:
#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0.0,Health Food Store,Trail,Pub,Neighborhood,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
1,East Toronto,0.0,Bus Line,Business Service,Park,Grocery Store,Discount Store,Yoga Studio,Elementary School,Flower Shop,Fish Market,Fish & Chips Shop
2,East Toronto,0.0,Fast Food Restaurant,Sandwich Place,Park,Liquor Store,Sushi Restaurant,Restaurant,Italian Restaurant,Pub,Steakhouse,Food & Drink Shop
3,East Toronto,0.0,Baseball Field,Athletics & Sports,Government Building,Night Market,Electronics Store,Food,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant
5,Central Toronto,0.0,Food & Drink Shop,Bus Line,Breakfast Spot,Department Store,Park,Gym,Dog Run,Ethiopian Restaurant,Food,Flower Shop
7,Central Toronto,0.0,Dessert Shop,Pizza Place,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Thai Restaurant,Diner,Restaurant,Seafood Restaurant
9,Central Toronto,0.0,Light Rail Station,Coffee Shop,Health & Beauty Service,Skating Rink,Café,Athletics & Sports,Supermarket,Liquor Store,Ethiopian Restaurant,Flower Shop
11,Downtown Toronto,0.0,Coffee Shop,Pub,Italian Restaurant,Park,Café,Pizza Place,Bakery,Restaurant,Farmers Market,Flower Shop
12,Downtown Toronto,0.0,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Pub,Grocery Store,Dance Studio,Gastropub,Hotel,Gay Bar
13,Downtown Toronto,0.0,Pub,Mediterranean Restaurant,Athletics & Sports,Café,Thai Restaurant,Seafood Restaurant,Mexican Restaurant,Food Truck,French Restaurant,Chocolate Shop


In [0]:
#Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,1.0,Furniture / Home Store,Park,Yoga Studio,Electronics Store,Food,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [0]:
#Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Toronto,2.0,Playground,Gym Pool,Park,Garden,Yoga Studio,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
10,Downtown Toronto,2.0,Playground,Park,Grocery Store,Candy Store,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [0]:
#Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,3.0,Construction & Landscaping,Bus Line,Swim School,Yoga Studio,Electronics Store,Food,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant


In [0]:
#Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,4.0,Convenience Store,Gym,Park,Yoga Studio,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
